<img src="http://sct.inf.utfsm.cl/wp-content/uploads/2020/04/logo_di.png" style="width:60%">

<center>
    <h1>ILI285/INF285 Computación Científica </h1>
    <h1>Pauta Pregunta de "Interpolación Zeta de Riemann" - COP3</h1>
</center>

La función zeta de Riemann se define de la siguiente forma: $\zeta(s)=\displaystyle\sum_{k=1}^\infty k^{-s}$, pero por simplicidad trabajaremos con la siguiente versión truncada $\zeta_m(s)=\displaystyle\sum_{k=1}^m k^{-s}$, para $m\gg 1$. Aún en su versión truncada, el costo de evaluarla es significativo. Considerando que se necesita obtener un error $\displaystyle\max_{s\in [2,10]} |\zeta_m(s)-p(s)|<\text{TOL}\in\{10^{-3}, 10^{-8}, 10^{-12}, 10^{-14}\}$, para $m\in\{100,200,300\}$, determine:
\{el grado mínimo del polinomio de interpolación utilizando puntos equiespaciados, el grado mínimo del polinomio de interpolación utilizando puntos de Chebyshev, la cantidad de intervalos en una Spline cúbica, con condiciones not a knot\}.

para que se cumpla la tolerancia del error requerida. 

Para calcular el error considere una grilla fina para $s$ de $500$ puntos equiespaciados y considere como número máximo de puntos de interpolación $N=70$. Si no se cumple la tolerancia solicitada con el máximo número de puntos indicados, su respuesta debe ser $-1$.

Utilice la implementación {```BarycentricInterpolator```, ```CubicSpline```} del paquete ```scipy.interpolate``` para realizar las interpolaciones. 

**Warning: El algoritmo solicitado está asociado a la tolerancia, por lo cual al elegir la tolerancia se define que algoritmo se debe utilizar.**

# Propuesta solución

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import BarycentricInterpolator, CubicSpline
from ipywidgets import interact, Dropdown
import ipywidgets as widgets

Defnición de $\zeta_m(s)$

In [2]:
# Truncated Zeta function
zeta = lambda m, s: np.sum(np.arange(1, m + 1, dtype=np.float) ** (-s))
zeta = np.vectorize(zeta)

Cálculo del error.

In [3]:
error = lambda z, p: np.max(np.abs(z - p))

Puntos de Chebyshev.

In [4]:
def cheb(a, b, n):
    i = np.arange(1, n + 1)
    x_i = np.cos((2 * i - 1) * np.pi / (2 * n)) # Chebyshev nodes
    return (b + a) / 2 + (b - a) * x_i / 2 # Change of variable for domain [a, b]

## Experimentos

Para distintos valores de $m$ y $TOL$.

In [5]:
def experiment(tol, m, opt, n_max=70, N_E=500):
    a, b = 2, 10 # Domain
    N = np.arange(2, n_max + 1)
    s = np.linspace(a, b, N_E) # Evaluation grid
    min_n = 0
    
    # Interpolation option, polynomial 1 and 2, spline 3
    inter = BarycentricInterpolator if opt < 3 else CubicSpline
    
    # Used only to avoid warning message (divide by zero) in polynomial interpolation
    np.seterr(divide='ignore') 
        
    for i in range(N.shape[0]):
        n = N[i]
        # Interpolation points, equispaced 1 or 3, Chebyshev 2
        s_i = cheb(a, b, n) if opt == 2 else np.linspace(a, b, n) 
        y_i = zeta(m, s_i)
        p = inter(s_i, y_i)

        if error(zeta(m, s), p(s)) < tol:
            min_n = n
            break
        
    # Polynomial degree or number of segments.
    # if N is n, polynomial degree or Spline segments are n-1. If any reach the TOL n is -1
    return min_n - 1

## Combinaciones para COP-3

In [6]:
TOL = [(1, 1e-8), (1, 1e-12), (2, 1e-14), (3, 1e-3)]
M = [100, 200, 300]
OPT = [
    "Para m=%d y TOL=%.e, el grado del polinomio utilizando puntos equispaciados es %d",
    "Para m=%d y TOL=%.e, el grado del polinomio utilizando puntos de Chebyshev es %d",
    "Para m=%d y TOL=%.e, el número de segmentos de la spline es %d"
]

In [7]:
def solution(o, m):
    opt, tol = TOL[o]
    r = experiment(tol, m, opt)
    print(OPT[opt-1]%(m, tol, r))
    
interact(solution,
        o=Dropdown(
            options=[(1e-3, 3), (1e-8, 0), (1e-12, 1), (1e-14, 2)],
            value=0,
            description='Tolerancia:'
        ),
        m=M
)

interactive(children=(Dropdown(description='Tolerancia:', index=1, options=((0.001, 3), (1e-08, 0), (1e-12, 1)…

<function __main__.solution(o, m)>